# Setting Up Environment

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

import time
import pandas as pd
import re
from fuzzywuzzy import fuzz
from bs4 import BeautifulSoup
import datetime as dt
import numpy as np

/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Fixing Bet Recommender

In [16]:
# Adding new fights to odds data
data_filled = pd.read_csv('mma_data_odds.csv', index_col = 0)
data_all = pd.read_csv('mma_data.csv', index_col = 0)
data_all = data_all[data_all.result >= 0]
data_all['Fighter_1_Odds'] = 0
data_all['Fighter_2_Odds'] = 0
# Filling odds w/ data with recent fights
last_row_filled = data_filled.tail(1)
fighter_1_last = last_row_filled.fighter_1.values[0]
fighter_2_last = last_row_filled.fighter_2.values[0]
data_all_copied = data_all.copy()
data_all_copied.reset_index(inplace = True, drop = True)
cutoff_unfilled = data_all_copied[(data_all_copied.fighter_1 == fighter_1_last) & 
                                (data_all_copied.fighter_2 == fighter_2_last)].index[0]
data_all_new = data_all_copied.iloc[cutoff_unfilled+1:]
data = pd.concat([data_filled, data_all_new])

# Filling in odds
driver = webdriver.Safari()
driver.get('https://www.bestfightodds.com/archive')
time.sleep(1)
for index, row in data.iterrows():
    try:
        if row.Fighter_1_Odds == 0:
            # Formatting name of higher ranked fighter
            fighter_1 = str(row.fighter_1)
            fighter_1 = re.findall('[A-Z][^A-Z]*', fighter_1)    
            fighter_name = ''
            for name in fighter_1:
                fighter_name = fighter_name + ' ' + name
            # Formatting name of lower ranked fighter
            fighter_2 = str(row.fighter_2)
            fighter_2 = re.findall('[A-Z][^A-Z]*', fighter_2)    
            fighter_name_2 = ''
            for name in fighter_2:
                fighter_name_2 = fighter_name_2 + ' ' + name
            # Searching for fights w/ higher ranked fighter
            search_bar = driver.find_elements(By.XPATH, '//*[@id="page-content"]/form/p/input[1]')[0]
            search_bar.send_keys(fighter_name)
            driver.find_elements(By.XPATH, '//*[@id="page-content"]/form/p/input[2]')[0].click()
            time.sleep(1)
            # Clicking on fighter 1 
            try:
                click_button = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="page-content"]/table[1]/tbody/tr[1]/td[2]/a')))
                driver.find_elements(By.XPATH, '//*[@id="page-content"]/table[1]/tbody/tr[1]/td[2]/a')[0].click()
                time.sleep(1)
            except:
                pass
            # Getting odds
            html = driver.page_source
            table = pd.read_html(html)[0]
            table = table[['Matchup', 'Closing range']]
            table['Fuzzy_1'] = table.Matchup.apply(lambda x: fuzz.ratio(x, fighter_name))
            table['Fuzzy_2'] = table.Matchup.apply(lambda x: fuzz.ratio(x, fighter_name_2))
            table = table[(table.Fuzzy_2 > 50) | (table.Fuzzy_1 > 50)].reset_index(drop = True)
            index_opp = table[table.Fuzzy_2 > 50].index[0]
            table_matchup = table.loc[index_opp-1:index_opp, :].reset_index(drop = True)
            # Filling odds
            data.loc[index, 'Fighter_1_Odds'] = table_matchup.loc[0, 'Closing range']
            data.loc[index, 'Fighter_2_Odds'] = table_matchup.loc[1, 'Closing range']
            # Navigating back and clearing text box
            driver.back()
            driver.implicitly_wait(10)
            driver.back()
            driver.implicitly_wait(10)
            driver.find_elements(By.XPATH, '//*[@id="page-content"]/form/p/input[1]')[0].clear()
        else:
            pass
    except:
        driver.quit()
        time.sleep(1)
        driver = webdriver.Safari()
        driver.get('https://www.bestfightodds.com/archive')
data = data[(data.Fighter_1_Odds != 0) & (data.Fighter_2_Odds != 0)]
data.dropna(subset = ['Fighter_1_Odds', 'Fighter_2_Odds'], inplace = True)
data.reset_index(inplace = True, drop = True)

# Saving updated file
data.to_csv('mma_data_odds.csv')

driver.quit()

In [18]:
data.tail(10)

,fighter_1,weight_1,reach_1,age_1,slpm_1,sapm_1,td_avg_1,sub_avg_1,strk_acc_1,strk_def_1,...,strk_def_2,td_acc_2,td_def_2,wins_2,losses_2,result,SUB_OVR,KO_OVR,Fighter_1_Odds,Fighter_2_Odds
722,PaulCraig,185.0,76.0,36,2.44,2.88,1.81,1.7,46.0,47.0,...,54.0,36.0,41.0,23.0,5.0,1,0,1,+175,-240
723,JaiHerbert,155.0,77.0,35,2.55,2.92,0.25,0.0,42.0,45.0,...,66.0,25.0,70.0,13.0,4.0,0,0,0,+120,-185
724,LeroneMurphy,145.0,73.0,32,3.58,2.64,0.83,0.6,48.0,55.0,...,59.0,0.0,82.0,11.0,1.0,1,0,0,-162,+120
725,DaveyGrant,135.0,69.0,38,4.78,3.89,1.41,0.2,44.0,54.0,...,71.0,0.0,88.0,14.0,0.0,0,0,0,0.0,0.0
726,DannyRoberts,170.0,74.0,36,3.08,3.50,0.26,0.3,40.0,51.0,...,55.0,0.0,50.0,8.0,2.0,0,0,1,-155,+110
727,MarcDiakiese,155.0,10.0,30,2.73,2.63,3.33,0.0,40.0,53.0,...,49.0,0.0,0.0,19.0,3.0,0,1,0,+160,-225
728,MickParkin,265.0,79.0,28,3.59,8.21,5.38,7.7,53.0,30.0,...,58.0,50.0,66.0,10.0,3.0,1,0,0,+115,-160
729,MakhmudMuradov,185.0,75.0,33,4.35,2.72,0.70,0.2,42.0,60.0,...,44.0,25.0,54.0,18.0,10.0,1,0,0,-400,+270
730,ChrisDuncan,155.0,10.0,30,4.69,4.69,4.73,0.0,52.0,41.0,...,71.0,0.0,0.0,7.0,0.0,1,0,0,-150,+110
731,JafelFilho,125.0,68.0,30,1.46,1.92,1.15,1.7,36.0,46.0,...,53.0,31.0,0.0,16.0,5.0,1,1,0,-125,-115
